# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


In [1]:
import requests
from bs4 import BeautifulSoup as bs

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)
soup = bs(r.text, 'html5lib')

### ① 印出最新文章的「作者」「標題」「時間」

In [9]:
soup

<!DOCTYPE html>
<html><head>
		<meta charset="utf-8"/>
		

<meta content="width=device-width, initial-scale=1" name="viewport"/>

<title>看板 NBA 文章列表 - 批踢踢實業坊</title>

<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>




	</head>
    <body>
		
<div id="topbar-container">
	<div class="bbs-content" id="topbar">
		<a href="/bbs/" id="logo">批踢踢實業坊</a>
		<span>›</span>
		<a class="board" href="/bbs/NBA/index.html"><span class="board-label">看板 </span>NBA</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">聯絡資訊</a>

In [51]:
# 試探：連進去子連結
bs(requests.get('https://www.ptt.cc'+soup.find_all(class_='title')[0].find('a')['href']).text, 'html5lib')

<!DOCTYPE html>
<html><head>
		<meta charset="utf-8"/>
		

<meta content="width=device-width, initial-scale=1" name="viewport"/>

<title>[情報] Kareem Abdul-Jabbar: 現役最想跟LBJ搭檔 - 看板 NBA - 批踢踢實業坊</title>
<meta content="all" name="robots"/>
<meta content="Ptt BBS 批踢踢" name="keywords"/>
<meta content="消息來源：
https://twitter.com/kaj33/status/1242258589932081152
great question!  Without a doubt Lebron, and The Lakers would be the right
team.
名人堂傳奇球星 Kareem Abdul-Jabbar
" name="description"/>
<meta content="Ptt 批踢踢實業坊" property="og:site_name"/>
<meta content="[情報] Kareem Abdul-Jabbar: 現役最想跟LBJ搭檔" property="og:title"/>
<meta content="消息來源：
https://twitter.com/kaj33/status/1242258589932081152
great question!  Without a doubt Lebron, and The Lakers would be the right
team.
名人堂傳奇球星 Kareem Abdul-Jabbar
" property="og:description"/>
<link href="https://www.ptt.cc/bbs/NBA/M.1585047813.A.D1D.html" rel="canonical"/>

<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>

In [52]:
# 試探：在子連結裡再找class
bs(requests.get('https://www.ptt.cc'+soup.find_all(class_='title')[0].find('a')['href']).text, 'html5lib')\
.find_all(class_='article-meta-value')

[<span class="article-meta-value">thnlkj0665 (灰色地帶)</span>,
 <span class="article-meta-value">NBA</span>,
 <span class="article-meta-value">[情報] Kareem Abdul-Jabbar: 現役最想跟LBJ搭檔</span>,
 <span class="article-meta-value">Tue Mar 24 19:03:29 2020</span>]

In [69]:
new = []
for x in soup.find_all(class_='title'):
    r = requests.get('https://www.ptt.cc'+x.find('a')['href'])
    s = bs(r.text, 'html5lib')
    
    temp = []
    try: #跳過爬公告文造成錯誤
        temp.append(s.find_all(class_="article-meta-value")[0].text)
        temp.append(s.find_all(class_="article-meta-value")[-2].text)
        temp.append(s.find_all(class_="article-meta-value")[-1].text)
        new.append(temp)
    except:
        continue

In [70]:
import pandas as pd

new = pd.DataFrame(new, columns=['作者', '標題', '時間'])
new['時間'] = pd.to_datetime(new['時間'])
new = new.sort_values(by='時間')
new.iloc[-1]

,作者,標題,時間
10,qazwsx879345 (Rajon Rondo),[公告] 版主上任相關事項,2019-10-25 10:44:03
11,Acetoxy (阿斯),[情報] SEASON Schedule 賽程 March 19–20,2020-03-02 01:19:40
12,Vedan (味丹),[公告] 第一次被退文，可在三天後刪除退文,2020-03-09 11:44:26
0,thnlkj0665 (灰色地帶),[情報] Kareem Abdul-Jabbar: 現役最想跟LBJ搭檔,2020-03-24 19:03:29
1,Aotearoa (長白雲之鄉),Fw: [影片] 2005 歐錦賽 德國 vs. 西班牙 Full Game,2020-03-24 19:11:13
2,hass231470 (),Re: [花邊] 感嘆大歐西進洛城 哈德威:續留魔術肯定奪冠,2020-03-24 19:15:12
3,ghost069 (阿鬼),[情報] NBA官方回顧16總冠軍賽G5:詹姆斯 歐,2020-03-24 20:10:54
4,rayisgreat (在過往中無盡徘徊),[新聞] 誰最接近上古神獸? 一眉哥被賈霸欽點,2020-03-24 20:26:05
5,Kazmier (代理人),[新聞] 林書豪回中國又發聲「叫武漢肺炎是種族歧,2020-03-24 21:34:22
6,fortun795 (未來幸運星),[花邊] 居家隔離鍛練M型化 訓練師：收入不夠高,2020-03-24 23:33:23


In [71]:
new.iloc[-1]

作者      derekhsu (華麗的天下無雙)
標題    [情報] 七六人教練團與制服組減薪20%
時間     2020-03-25 00:31:14
Name: 9, dtype: object

### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [73]:
url = 'https://www.ptt.cc/bbs/NBA/index1.html'
r = requests.get(url)
soup = bs(r.text, 'html5lib')

first = []
for x in soup.find_all(class_='title'):
    r = requests.get('https://www.ptt.cc'+x.find('a')['href'])
    s = bs(r.text, 'html5lib')
    
    temp = []
    temp.append(s.find_all(class_='article-meta-value')[0].text)
    temp.append(s.find_all(class_='article-meta-value')[-2].text)
    temp.append(pd.to_datetime(s.find_all(class_='article-meta-value')[-1].text))
    first.append(temp)

pd.DataFrame(first, columns=['作者', '標題', '時間'])

,作者,標題,時間
0,Price (Price),[轉錄]Lyotard 對於太陽板討論串的結論,2004-05-16 01:54:33
1,Price (Price),[公告] 請停止一切關於本次活動的發言,2004-05-24 00:43:00
2,Frankaze (神采風華),[轉錄]跟之前那篇比起來 我覺得這篇也應該轉過來,2004-06-08 01:16:45
3,Frankaze (神采風華),[轉錄]再轉一篇好文來,2004-06-08 09:02:23
4,Price (Price),[轉錄][情報] 夏洛特山貓系列,2004-06-10 00:39:35
5,Price (Price),[轉錄]Re: [心得] 真是受不了糗爺....,2004-06-10 00:51:35
6,Price (Price),[轉錄]總冠軍賽NO.2觀後感,2004-06-10 12:05:07
7,AmuroNamie (原來太嗨真的會失眠^___^),[心得] Rasheed Wallace,2004-06-11 15:06:09
8,toptree ( ),掌控球賽的男人,2004-06-12 07:13:04
9,skchang (3EB板開板囉！),[閒聊] 說說2004季後賽名場面回顧,2004-06-12 13:21:49


### ③ 試著爬爬看其他版的文章

In [93]:
url = 'https://www.ptt.cc/bbs/sex/index.html'

r = requests.get(url, cookies={'over18': '1'})
soup = bs(r.text, 'html5lib')
requests.get('https://www.ptt.cc'+soup.find_all(class_='title')[0].find('a')['href'], cookies={'over18': '1'})

<Response [200]>

In [105]:
url = 'https://www.ptt.cc/bbs/sex/index.html'

r = requests.get(url, cookies={'over18': '1'})
soup = bs(r.text, 'html5lib')

new = []
for x in soup.find_all(class_='title'):
    try: #往前判斷，避免爬到被刪文的錯誤連結
        r = requests.get('https://www.ptt.cc'+x.find('a')['href'], cookies={'over18': '1'})
        s = bs(r.text, 'html5lib')
    
        temp = []
        temp.append(s.find_all(class_="article-meta-value")[0].text)
        temp.append(s.find_all(class_="article-meta-value")[-2].text)
        temp.append(s.find_all(class_="article-meta-value")[-1].text)
        new.append(temp)
    except:
        continue

pd.DataFrame(new, columns=['作者', '標題', '時間'])

,作者,標題,時間
0,Whoootz (呼呼),Re: [閒聊] 有男生被吃過豆腐嗎？,Tue Mar 24 18:03:55 2020
1,inshadow (硬射抖),Re: [閒聊] 有男生被吃過豆腐嗎？,Tue Mar 24 19:01:50 2020
2,zyc5566 (和風蒜頭雞湯),Re: [閒聊] 說不約的女生是真的不約嗎,Tue Mar 24 19:38:40 2020
3,hvariables (Speculative Male),[新聞] 花5700元與國中妹上床 爽完遭求償889萬,Tue Mar 24 19:58:58 2020
4,MaskedWolf (睡紙箱的說書人),[創作] 白濁象牙塔 (10),Tue Mar 24 20:09:59 2020
5,dapurper (不是你的天使),[閒聊] 有女生也喜歡看奶照的嗎？,Tue Mar 24 20:31:08 2020
6,ytkuang (就是那個光),[新聞] 泥醉美眉摩鐵裸睡「下體痛」驚醒 色男指,Tue Mar 24 20:48:28 2020
7,etshaw (小萌牛),[討論] 不知道怎麼達成男友的請求,Tue Mar 24 21:37:18 2020
8,sunbear928 (小熊一個),[閒聊] 為何A片中的男生可以在其他男生前勃起,Tue Mar 24 22:09:40 2020
9,rukawa28 (壽豐臣秀吉 是也),是盤子大盤商還是....,Tue Mar 24 22:55:38 2020
